The goal is to develop a linear regression model capable of predicting the annual spending of e-commerce customers using their data.

In [1]:
#Starting a spark session 

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('eCommerce_linear_regression_code').getOrCreate()

In [2]:
#import regression modules 

from pyspark.ml.regression import LinearRegression

In [3]:
#read csv file of ecommerce data 
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema = True, header = True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
#one row of this data frame contains 
for column in data.head():
    print(column)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
#copy the variables from data.columns

assembler = VectorAssembler(
    inputCols = [
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],
outputCol = 'features')

In [9]:
#transforming data with assambler 

output = assembler.transform(data)

In [10]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows


In [11]:
final_data = output.select("features","Yearly Amount Spent")
#Select only the columns needed for the model

In [12]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [13]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                363|
|   mean|  498.9929613909979|
| stddev|  78.28859768782743|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                137|
|   mean|  500.1647747763734|
| stddev|  82.25918670978157|
|    min| 256.67058229005585|
|    max|  689.7876041747194|
+-------+-------------------+



In [14]:
# crating linear regression model 
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [15]:
#fit the model to the data
lrModel = lr.fit(train_data)

In [17]:
# Print the coefficients and intercept for linear regression
# Coefficients (slope) → Show how much each independent variable (feature) influences the target variable.
# Intercept (y-intercept) → The predicted value of the target variable when all independent variables are zero.
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients , lrModel.intercept))

Coefficients: [25.894909615449045,38.92661668714369,0.3079099718095319,61.69883872216042] Intercept: -1055.8501943366462


In [19]:
# Evaluate the model on the test data
test_results = lrModel.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-15.668293939938962|
|  5.931884512719421|
| -6.796395425350738|
| 10.827845034991924|
|  22.59827762503278|
| 20.072155557309827|
|  4.659836629955521|
|-3.3757174457944075|
| 3.6862353030109603|
|-6.3101243391382695|
| -7.864030810325119|
|-17.248181870236976|
| 17.818248722904684|
|-3.7200765279747543|
| -1.398680583558587|
|   8.64345217199201|
|  8.922378637352779|
|-1.5441588474657237|
| -9.733319473629876|
|-16.438005686716735|
+-------------------+
only showing top 20 rows


In [20]:
unlabeled_data = test_data.select('features')

In [21]:
predictions = lrModel.transform(unlabeled_data)
# Make predictions using the trained model

In [22]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8162006488763...|281.75463488840796|
|[31.0472221394875...|  386.565514676302|
|[31.1280900496166...| 564.0490821724054|
|[31.1695067987115...| 416.5286857673009|
|[31.2834474760581...| 569.1828118006347|
|[31.3123495994443...| 443.5192624706308|
|[31.3584771924370...| 490.5161138195199|
|[31.4252268808548...| 534.1424361005563|
|[31.4459724827577...| 481.1907296321176|
|[31.4474464941278...| 424.9128664343623|
|[31.5261978982398...|416.95855700266293|
|[31.5702008293202...| 563.1936740116419|
|[31.6098395733896...| 426.7273009282035|
|[31.6253601348306...|380.05697728489895|
|[31.8186165667690...|447.81735395369424|
|[31.8209982016720...|416.03182884122134|
|[31.8512531286083...| 464.0698680294456|
|[31.8530748017465...| 460.8292823098177|
|[31.8648325480987...|449.62459995044355|
|[31.9048571310136...| 490.3878631095329|
+--------------------+------------

In [23]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))


RMSE: 10.620653738711452
MSE: 112.79828583760553
